In [0]:
import numpy as np
import pandas as pd
import gc

## Stuff below is to load the CSV's through google drive. Way too slow otherwise.

In [0]:
!pip install -U -q PyDrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
# Authenticate and create the PyDrive client.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [0]:
calendar_csv_drive_link = "https://drive.google.com/open?id=11FCINUYChtNJHRyY1mzbhaDWgVwlJzA9"
sales_train_validation_csv_drive_link = "https://drive.google.com/open?id=1CrJXm3TJBVoLwlG607kIFNCoxBPRem6C"
sample_submission_csv_drive_link = "https://drive.google.com/open?id=1F6ORVpIh5r-998HfGpv_jmitIQVc7d9j"
sell_prices_csv_drive_link = "https://drive.google.com/open?id=1BT-IveptPb3BjNcVt-sFWHabnrubwsGD"

calendar_id = calendar_csv_drive_link.split("=")[-1]
sales_train_val_id = sales_train_validation_csv_drive_link.split("=")[-1]
sample_submission_id = sample_submission_csv_drive_link.split("=")[-1]
sell_prices_id = sell_prices_csv_drive_link.split("=")[-1]

downloaded_calendar = drive.CreateFile({'id': calendar_id})
downloaded_calendar.GetContentFile('calendar.csv')

downloaded_sales_train = drive.CreateFile({'id': sales_train_val_id})
downloaded_sales_train.GetContentFile('sales_train_validation.csv')

downloaded_sales_train = drive.CreateFile({'id': sample_submission_id})
downloaded_sales_train.GetContentFile('sample_submission.csv')

downloaded_sales_train = drive.CreateFile({'id': sell_prices_id})
downloaded_sales_train.GetContentFile('sell_prices.csv')

Loading the data into DataFrames

In [25]:
sell_prices_df = pd.read_csv('sell_prices.csv')
sales_train_val_df = pd.read_csv('sales_train_validation.csv')
calendar_df = pd.read_csv('calendar.csv')
sell_prices_df.shape, sales_train_val_df.shape

((6841121, 4), (30490, 1919))

In [26]:
sales_train_val_df.head()

,id,item_id,dept_id,cat_id,store_id,state_id,d_1,d_2,d_3,d_4,d_5,d_6,d_7,d_8,d_9,d_10,d_11,d_12,d_13,d_14,d_15,d_16,d_17,d_18,d_19,d_20,d_21,d_22,d_23,d_24,d_25,d_26,d_27,d_28,d_29,d_30,d_31,d_32,d_33,d_34,...,d_1874,d_1875,d_1876,d_1877,d_1878,d_1879,d_1880,d_1881,d_1882,d_1883,d_1884,d_1885,d_1886,d_1887,d_1888,d_1889,d_1890,d_1891,d_1892,d_1893,d_1894,d_1895,d_1896,d_1897,d_1898,d_1899,d_1900,d_1901,d_1902,d_1903,d_1904,d_1905,d_1906,d_1907,d_1908,d_1909,d_1910,d_1911,d_1912,d_1913
0,HOBBIES_1_001_CA_1_validation,HOBBIES_1_001,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,1,1,3,1,3,1,2,2,0,1,1,1,1,0,0,0,0,0,1,0,4,2,3,0,1,2,0,0,0,1,1,3,0,1,1,1,3,0,1,1
1,HOBBIES_1_002_CA_1_validation,HOBBIES_1_002,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,1,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0
2,HOBBIES_1_003_CA_1_validation,HOBBIES_1_003,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,1,2,2,1,2,1,1,1,0,1,1,1
3,HOBBIES_1_004_CA_1_validation,HOBBIES_1_004,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,1,3,4,2,1,4,1,3,5,0,6,6,0,0,0,0,3,1,2,1,3,1,0,2,5,4,2,0,3,0,1,0,5,4,1,0,1,3,7,2
4,HOBBIES_1_005_CA_1_validation,HOBBIES_1_005,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,1,0,3,2,2,2,3,1,0,0,0,0,1,0,4,4,0,1,4,0,1,0,1,0,1,1,2,0,1,1,2,1,1,0,1,1,2,2,2,4


In [27]:
object_columns = sales_train_val_df.select_dtypes('object')

print(object_columns)

                                  id        item_id  ... store_id state_id
0      HOBBIES_1_001_CA_1_validation  HOBBIES_1_001  ...     CA_1       CA
1      HOBBIES_1_002_CA_1_validation  HOBBIES_1_002  ...     CA_1       CA
2      HOBBIES_1_003_CA_1_validation  HOBBIES_1_003  ...     CA_1       CA
3      HOBBIES_1_004_CA_1_validation  HOBBIES_1_004  ...     CA_1       CA
4      HOBBIES_1_005_CA_1_validation  HOBBIES_1_005  ...     CA_1       CA
...                              ...            ...  ...      ...      ...
30485    FOODS_3_823_WI_3_validation    FOODS_3_823  ...     WI_3       WI
30486    FOODS_3_824_WI_3_validation    FOODS_3_824  ...     WI_3       WI
30487    FOODS_3_825_WI_3_validation    FOODS_3_825  ...     WI_3       WI
30488    FOODS_3_826_WI_3_validation    FOODS_3_826  ...     WI_3       WI
30489    FOODS_3_827_WI_3_validation    FOODS_3_827  ...     WI_3       WI

[30490 rows x 6 columns]


In [29]:

pd.get_dummies(sales_train_val_df, columns=["cat_id","store_id","dept_id","state_id"], 
               prefix=["category","store","dept","state"]).head()

,id,item_id,d_1,d_2,d_3,d_4,d_5,d_6,d_7,d_8,d_9,d_10,d_11,d_12,d_13,d_14,d_15,d_16,d_17,d_18,d_19,d_20,d_21,d_22,d_23,d_24,d_25,d_26,d_27,d_28,d_29,d_30,d_31,d_32,d_33,d_34,d_35,d_36,d_37,d_38,...,d_1897,d_1898,d_1899,d_1900,d_1901,d_1902,d_1903,d_1904,d_1905,d_1906,d_1907,d_1908,d_1909,d_1910,d_1911,d_1912,d_1913,category_FOODS,category_HOBBIES,category_HOUSEHOLD,store_CA_1,store_CA_2,store_CA_3,store_CA_4,store_TX_1,store_TX_2,store_TX_3,store_WI_1,store_WI_2,store_WI_3,dept_FOODS_1,dept_FOODS_2,dept_FOODS_3,dept_HOBBIES_1,dept_HOBBIES_2,dept_HOUSEHOLD_1,dept_HOUSEHOLD_2,state_CA,state_TX,state_WI
0,HOBBIES_1_001_CA_1_validation,HOBBIES_1_001,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,1,2,0,0,0,1,1,3,0,1,1,1,3,0,1,1,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0
1,HOBBIES_1_002_CA_1_validation,HOBBIES_1_002,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0
2,HOBBIES_1_003_CA_1_validation,HOBBIES_1_003,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,2,2,1,2,1,1,1,0,1,1,1,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0
3,HOBBIES_1_004_CA_1_validation,HOBBIES_1_004,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2,0,...,2,5,4,2,0,3,0,1,0,5,4,1,0,1,3,7,2,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0
4,HOBBIES_1_005_CA_1_validation,HOBBIES_1_005,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,1,1,2,0,1,1,2,1,1,0,1,1,2,2,2,4,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0


In [30]:
calendar_df.head()

,date,wm_yr_wk,weekday,wday,month,year,d,event_name_1,event_type_1,event_name_2,event_type_2,snap_CA,snap_TX,snap_WI
0,2011-01-29,11101,Saturday,1,1,2011,d_1,NaN,NaN,NaN,NaN,0,0,0
1,2011-01-30,11101,Sunday,2,1,2011,d_2,NaN,NaN,NaN,NaN,0,0,0
2,2011-01-31,11101,Monday,3,1,2011,d_3,NaN,NaN,NaN,NaN,0,0,0
3,2011-02-01,11101,Tuesday,4,2,2011,d_4,NaN,NaN,NaN,NaN,1,1,0
4,2011-02-02,11101,Wednesday,5,2,2011,d_5,NaN,NaN,NaN,NaN,1,0,1


In [31]:
sell_prices_df.head()

,store_id,item_id,wm_yr_wk,sell_price
0,CA_1,HOBBIES_1_001,11325,9.58
1,CA_1,HOBBIES_1_001,11326,9.58
2,CA_1,HOBBIES_1_001,11327,8.26
3,CA_1,HOBBIES_1_001,11328,8.26
4,CA_1,HOBBIES_1_001,11329,8.26


Checking out the dtypes - reducing the int64's to make it more memory efficient

In [32]:

calendar_df[["month", "snap_CA", "snap_TX", "snap_WI", "wday"]] = \
calendar_df[["month", "snap_CA", "snap_TX", "snap_WI", "wday"]].astype("int8")
calendar_df[["wm_yr_wk", "year"]] = calendar_df[["wm_yr_wk", "year"]].astype("int16") 
calendar_df["date"] = calendar_df["date"].astype("datetime64")
print(calendar_df.dtypes)

date            datetime64[ns]
wm_yr_wk                 int16
weekday                 object
wday                      int8
month                     int8
year                     int16
d                       object
event_name_1            object
event_type_1            object
event_name_2            object
event_type_2            object
snap_CA                   int8
snap_TX                   int8
snap_WI                   int8
dtype: object
